# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、チャットボットの応答の好みを予測する問題に取り組んでいます。具体的には、2つの異なる言語モデルが生成した応答のうち、どちらがユーザーに好まれるかを判定するモデルを構築することを目指しています。

### 主要な手法とライブラリ
- **ライブラリ**: 主に`pandas`（データ処理）、`torch`（深層学習処理）、および`transformers`（言語モデルの利用）を使用しています。
  
- **データ処理**: `process_data`関数を用いて、CSVファイルから読込んだプロンプトおよび応答を整形しています。このプロセスでは、前後の角括弧やダブルクオーテーションを取り除き、文章を結合しています。

- **モデルのロード**: `AutoTokenizer`と`AutoModelForCausalLM`を利用して、事前トレーニングされたGemma-2-9Bモデルをロードしています。モデルはbfloat16の形式でCUDAデバイスにロードされ、推論に対応しています。

- **推論処理**: `predict`関数を定義し、入力クエリと2つの応答を元にどちらの応答が優れているかを判断します。評価基準には関連性、正確性、完全性、一貫性が考慮されており、応答の判定結果に基づいて出力が行われます。

- **結果の蓄積**: ループ処理を通じて、全テストデータに対して予測を行い、その結果をリストにまとめ、最終的に`submission.csv`という形式で出力します。各エントリは、クエリID、モデルAの勝者、モデルBの勝者、および引き分けのフラグを含みます。

全体的には、このノートブックは機械学習を介して言語モデルの応答を評価し、人間の好みに基づいて最適な応答を予測するための手法を示しています。

---


# 用語概説 
以下に、Jupyter Notebookで使用されている専門用語に関する簡易解説を示します。初心者がつまずきやすいマイナーな用語や、特有のドメイン知識に焦点を当てています。

1. **bfloat16**: 
   - バイナリ浮動小数点数の一種で、16ビットの精度を持ちます。機械学習においては、メモリの使用量を削減し、計算効率の向上を図るために用いられます。特に、TPU（Tensor Processing Unit）でよく利用されるフォーマットです。

2. **プロンプトエンジニアリング**:
   - 大規模言語モデル（LLM）に対する入出力を最適化するための手法で、モデルに与える指示や質問（プロンプト）を慎重に設計することです。これにより、モデルから得られる応答の品質を向上させることが目的です。

3. **CUDA**:
   - NVIDIAが開発した、GPUを用いた計算を効率的に行うための並列計算プラットフォームおよびAPIです。機械学習や深層学習のトレーニングおよび推論において、GPUを利用するために広く用いられます。

4. **トークン化**:
   - 自然言語処理において、テキストを分析可能な単位（トークン）に分割するプロセスです。これは、モデルが理解できる形式に変換するための前処理の一環として行われます。

5. **生成モデル**:
   - 入力データを基に新しいデータを生成することを目的とする機械学習モデルの一種です。本ノートブックでは、特に言語生成に関与するモデル（例えば、文や会話の生成）を指します。

6. **メモリ効率の良いSDP (Stochastic Dynamic Programming)**:
   - モデルの計算を効率化する手法で、メモリの使用を抑えつつ動的プログラミングを行うものです。大規模なモデルのトレーニング時に特に重要です。

7. **重み**:
   - 機械学習モデルがデータから学習するパラメータのこと。重みは、モデルの性能に大きな影響を与え、トレーニングの過程で調整されます。

8. **次元削減**:
   - 多次元空間のデータを少ない次元に圧縮する手法で、データの可視化や処理の効率化に用いられます。具体的な手法にはPCA（主成分分析）やt-SNEが含まれますが、ノートブック内では直接触れられていないかもしれません。

9. **スニペット**:
   - プログラムコードの短い部分やフラグメントを示す用語で、通常は再利用可能なものを指します。コーディングの際によく用いられます。

10. **グラデーション降下法**:
    - 最適化アルゴリズムの一つで、誤差を最小化するためにモデルの重みを更新する方法です。勾配を計算し、その反対方向に重みを調整することで、より良いパラメータを見つけ出します。

これらの用語は、特に実務経験がない初心者にとっては馴染みが薄いか、理解が難しいことがあります。これらの解説が理解の助けとなれば幸いです。

---


In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import ast
import json
import re
import torch
from tqdm import tqdm

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率の良いSDPを無効にする
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にする

In [ ]:
def process_data(input_str):
    stripped_str = input_str.strip('[]')  # 文字列の前後の角括弧を削除する
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文章を分割し、各部分の前後のダブルクオーテーションを削除
    sentences = ' '.join(sentences)  # 分割した文章をスペースで結合
    return sentences

def get_data(path, system_prompt=None):
    df = pd.read_csv(path)  # 指定したパスからCSVファイルを読み込む
    df['prompt'] = df['prompt'].apply(process_data)  # プロンプト列に対してprocess_dataを適用
    df['response_a'] = df['response_a'].apply(process_data)  # 応答A列に対してprocess_dataを適用
    df['response_b'] = df['response_b'].apply(process_data)  # 応答B列に対してprocess_dataを適用
    return df  # 処理済みのデータフレームを返す

In [ ]:
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'  # テストデータのパスを指定
test_df = get_data(test_path)  # テストデータを取得
test_df.head()  # テストデータの最初の5行を表示

## Gemma-2-9Bモデルのロード
> google/gemma-2-9b-it 

私はすでにbfloat16の重みをダウンロードして保存しました。



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer  # Transformersライブラリからモデルとトークナイザーをインポート

In [ ]:
tokenizer_path = '/kaggle/input/gemma-2-9b-it/gemma-2-9b-it-palash-tokenizer'  # トークナイザーのパスを指定
model_path = '/kaggle/input/gemma-2-9b-it/gemma-2-9b-it-palash-model'  # モデルのパスを指定

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)  # トークナイザーを指定したパスからロード

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.bfloat16)  # モデルを指定したパスからロードし、デバイスマップを自動で取得、データ型をbfloat16に設定

In [ ]:
model  # モデルオブジェクトを表示

## シンプルな推論を行ってみましょう



In [ ]:
%%time
prompt = 'Write a conversation between gemma and llama llm models'  # プロンプトを定義
input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')  # プロンプトをトークン化し、テンソル形式に変換してCUDAデバイスに移動

outputs = model.generate(**input_ids, max_new_tokens=200)  # モデルを使って新しいトークンを生成
response = tokenizer.decode(outputs[0])  # 生成したトークンをデコードして応答を取得
print(response)  # 応答を表示

## テストセットに対して、プロンプトエンジニアリングを行った推論を行いましょう



In [ ]:
test_df  # テストデータフレームを表示

In [ ]:
def get_prompt(query, response_a, response_b):
    prompt = f"""
あなたは、異なるモデルによって生成された2つの応答を評価し、どちらの方が優れているかを判断する任務を負っています。クエリと2つの応答（モデルAからの応答AとモデルBからの応答B）を与えられた場合、各応答の質を関連性、正確性、完全性、全体的な一貫性に基づいて評価します。両方の応答が同等に良いか悪い場合は、引き分けと宣言することができます。

指示：

クエリ: {query}
応答A（モデルA）: {response_a}
応答B（モデルB）: {response_b}

評価基準：

関連性: 応答はクエリにどれだけ対応しているか？
正確性: 提供された情報は正確で信頼性があるか？
完全性: 応答は包括的な回答を提供しているか？
一貫性: 応答は論理的に構成されており、理解しやすいか？
出力：

応答Aが優れている場合、出力: 応答A
応答Bが優れている場合、出力: 応答B
両方の応答が同等に良いまたは悪い場合、出力: 引き分け

単一行を出力する必要があります – 応答Aまたは応答Bまたは引き分けのいずれかの単語\n
出力: 

    """
    return prompt

In [ ]:
def predict(query, response_a, response_b, max_new_tokens=50, do_sample=False, temperature=1.0):
    prompt = get_prompt(query=query, response_a=response_a, response_b=response_b)

    input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')  # プロンプトをトークン化し、テンソル形式に変換してCUDAデバイスに移動

    outputs = model.generate(**input_ids, max_new_tokens=50, do_sample=do_sample, temperature=temperature)  # モデルが新しいトークンを生成
    response = tokenizer.decode(outputs[0])  # 生成したトークンをデコードして応答を取得

    pattern = r"OUTPUT:\s*(RESPONSE_A|RESPONSE_B|TIE)"  # 出力のパターンを定義
    match = re.search(pattern, response)  # 応答の中から出力パターンを検索
    if match:
        pred = match.group(1)  # マッチした場合、出力を取得
    else:
        pred = None  # マッチしなかった場合はNoneを設定
    return pred  # 予測結果を返す

In [ ]:
%%time
id_list = []
winner_model_a_list = []
winner_model_b_list = []
winner_tie_list = []
for idx in tqdm(range(0, len(test_df))):  # テストデータフレームの各行についてループ
    query_id = test_df.iloc[idx]['id']  # クエリIDを取得
    query = test_df.iloc[idx]['prompt']  # クエリを取得
    response_a = test_df.iloc[idx]['response_a']  # 応答Aを取得
    response_b = test_df.iloc[idx]['response_b']  # 応答Bを取得
    pred = predict(query, response_a, response_b, max_new_tokens=20, do_sample=True, temperature=0.7)  # 予測を実行
    id_list.append(query_id)  # IDをリストに追加
    if pred is not None:  # 予測結果がNoneでない場合
        if 'A' in pred or 'a' in pred:  # 応答Aが優れている場合
            winner_model_a_list.append(1)  # 応答Aの勝者リストに1を追加
            winner_model_b_list.append(0)  # 応答Bの勝者リストに0を追加                    
            winner_tie_list.append(0)  # 引き分けリストに0を追加
        if 'B' in pred or 'b' in pred:  # 応答Bが優れている場合
            winner_model_a_list.append(0)  # 応答Aの勝者リストに0を追加
            winner_model_b_list.append(1)  # 応答Bの勝者リストに1を追加        
            winner_tie_list.append(0)  # 引き分けリストに0を追加            
    else:
        winner_model_a_list.append(0)  # 応答Aの勝者リストに0を追加
        winner_model_b_list.append(0)  # 応答Bの勝者リストに0を追加        
        winner_tie_list.append(1)  # 引き分けリストに1を追加
    torch.cuda.empty_cache()  # CUDAメモリのキャッシュをクリア

In [ ]:
submission_df = pd.DataFrame({'id': id_list, 'winner_model_a': winner_model_a_list, 'winner_model_b': winner_model_b_list, 'winner_tie': winner_tie_list})  # 提出用のデータフレームを作成

In [ ]:
submission_df  # 提出用データフレームを表示

In [ ]:
submission_df.to_csv('submission.csv', index=False)  # 提出データフレームをCSVファイルとして保存